In [ ]:
!pip install pylandstats

In [2]:
import pylandstats as pls
import os,glob,time
import numpy as np
import rasterio as rio
import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import time
import logging

%matplotlib inline

print(os.getcwd())

# Globals

proj = 'EPSG:5070'

#################
# Load the data #
#################

# # Spatial blocks
# blocks_path = os.path.join(projdir,'spatial/mod/boundaries/spatial_block_grid_50km2_count_s2.gpkg')
# blocks = gpd.read_file(blocks_path).to_crs(proj)
# blocks['s2aspen_sum'] = blocks['s2aspen_sum'].astype(int)
# blocks = blocks[blocks['s2aspen_sum'] > 100000]  # at least 10 hectares
# block_ids = list(blocks.id)
# print(len(block_ids))

# # White River NF boundary
# wrnf_path = os.path.join(projdir,'spatial/raw/boundaries/wrnf_boundary_srme.gpkg')

# Target grids (resampled to 10-meter where necessary)
tifs = [
    's2aspen_prob_10m_binOpt_wrnf.tif',
    'lc16_evt_200_bin_wrnf_10m.tif',
    'usfs_treemap16_balive_int_bin_wrnf_10m.tif',
    'usfs_itsp_aspen_ba_gt10_wrnf_10m.tif'
]

print("Success ...")

/home/jovyan
Success ...


In [3]:
"""
Define a function to calculate landscape patch metrics given an input landscape surface
"""

def compute_ls_metrics(landscape,source='',roi='',blockId='',class_code=1):
    
    # start_time = time.time()
            
    # Compute the patch metrics
    area = pd.DataFrame(landscape.area(class_val=class_code, hectares=True))  # patch area
    perimeter = pd.DataFrame(landscape.perimeter(class_val=class_code)) # perimeter length    
    perimeter_ar_r = pd.DataFrame(landscape.perimeter_area_ratio(class_val=class_code, hectares=True))
    shp_i = pd.DataFrame(landscape.shape_index(class_val=class_code))
    
    patch_df = pd.concat([area,perimeter,perimeter_ar_r,shp_i], axis=1)  # concatenate into a single data frame
    
    patch_df['region'] = roi
    patch_df['source'] = source

    del area, perimeter, perimeter_ar_r, shp_i  # free up space

    # Class/landscape-level metrics
    tot_ar = landscape.total_area(class_val=class_code, hectares=True)
    prop = landscape.proportion_of_landscape(class_val=class_code, percent=True)
    n_ptchs = landscape.number_of_patches(class_val=class_code)
    ptch_den = landscape.patch_density(class_val=class_code, percent=True, hectares=True)

    class_df = pd.DataFrame(
        {'total_area': [tot_ar],
         'prop_area': [prop],
         'n_patch': [n_ptchs],
         'patch_den': [ptch_den]
        }
    )
    class_df['region'] = roi
    class_df['source'] = source

    # Set the block ID column if needed (for the Southern Rockies)
    if blockId != '':
        patch_df['block_id'] = blockId
        class_df['block_id'] = blockId
        
    # Free up some memory
    del tot_ar, prop, n_ptchs, ptch_den

    # print(f"Time elapsed: {round(time.time() - start_time, 1)} seconds.")

    return patch_df, class_df

print("Ready")

Ready


In [4]:
"""
Loop through the reference images (both Sentinel-based map and existing products)
Generate the landscape surface, calculate patch and landscape level statistics
For Southern Rockies, analyze by spatial block
"""

rois = ['srme','wrnf']

start = time.time()

# out_dir = os.path.join(projdir,'tabular/mod/results/ls_metrics/')

for region in rois:

    # Grab a list of images
    tif_paths = [tif for tif in tifs if str(region) in tif]

    if region == 'srme':
        print("Processing by spatial block for the Southern Rockies ...")
        print([os.path.basename(tif)[:-4] for tif in tif_paths])
        
        for tif_path in tif_paths:

            # Grab the name of the image file being processed
            source_name = os.path.basename(tif_path)[:-4]
            print(f"Starting: {source_name}")
            
            # Check if the file exists, if not, proceed
            if not os.path.exists(f'ls_metrics_patch_{source_name}.csv'):

                start_tif = time.time()
                
                # Open the image file
                img = rxr.open_rasterio(tif_path, masked=True, cache=False).squeeze()
                img = img.fillna(0)  # fill the nodata value to 0
                print(f"NoData value: {img.rio.nodata}")
                    
                block_patch_results = []  # empty list to store patch results
                block_class_results = []  # empty list to store class results

                # Loop through block IDs, calculate the metrics
                for bid in tqdm(block_ids, desc='Processing blocks'):
                    
                    # Grab the block geometry
                    block = blocks[blocks['id'] == bid].geometry
    
                    # Generate the landscape surface
                    clipped = img.rio.clip(block)  # clip to the block
                    ls = pls.Landscape(clipped.values, res=(10,10))  # Convert to landscape object
                    
                    del block, clipped  # free up some memory
    
                    # Compute the landscape metrics
                    try:
                        patch_results, class_results = compute_ls_metrics(ls, source_name, region, bid)
                    except Exception as e: 
                        logging.error(f"Failed to compute metrics for block {bid} in {region}: {e}")
                        continue
                    
                    block_patch_results.append(patch_results)
                    block_class_results.append(class_results)
    
                    del ls, patch_results, class_results  # free up memory
    
                del img 
                
                # Bind the block metrics together
                patch_results = pd.concat(block_patch_results, axis=0)
                class_results = pd.concat(block_class_results, axis=0)
    
                # Save out to CSV
                    
                patch_results.to_csv(os.path.join(out_dir,f'ls_metrics_patch_{source_name}.csv'))
                class_results.to_csv(os.path.join(out_dir,f'ls_metrics_class_{source_name}.csv'))
    
                del patch_results, class_results
    
                print(f"Time elapsed: {round((time.time() - start_tif)/60, 2)} minutes.")
    
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    else:

        print("Processing for the White River NF ...")

        for tif_path in tif_paths:

            print([os.path.basename(tif)[:-4] for tif in tif_paths])

            # Grab the name of the image file being processed
            source_name = os.path.basename(tif_path)[:-4]
            print(f"Starting: {source_name}")
            
            # Check if the file exists, if not, proceed
            if not os.path.exists(f'ls_metrics_patch_{source_name}.csv'):
                # Open the image file
                img = rxr.open_rasterio(tif_path, masked=True, cache=False).squeeze()
                img = img.fillna(0)
                print(f"NoData value: {img.rio.nodata}")
                
                # Generate the landscape surface
                ls = pls.Landscape(img.values, res=(10,10))  # Convert to landscape object
        
                # Compute the landscape metrics
                patch_results, class_results = compute_ls_metrics(ls, source_name, region)
        
                patch_results.to_csv(f'ls_metrics_patch_{source_name}.csv')
                class_results.to_csv(f'ls_metrics_class_{source_name}.csv')
        
                del img, ls, patch_results, class_results
    
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                

print(f"Total elapsed time: {round(time.time() - start, 1)/60} minutes.")


Processing by spatial block for the Southern Rockies ...
[]
Processing for the White River NF ...
['s2aspen_prob_10m_binOpt_wrnf', 'lc16_evt_200_bin_wrnf_10m', 'usfs_treemap16_balive_int_bin_wrnf_10m', 'usfs_itsp_aspen_ba_gt10_wrnf_10m']
Starting: s2aspen_prob_10m_binOpt_wrnf
['s2aspen_prob_10m_binOpt_wrnf', 'lc16_evt_200_bin_wrnf_10m', 'usfs_treemap16_balive_int_bin_wrnf_10m', 'usfs_itsp_aspen_ba_gt10_wrnf_10m']
Starting: lc16_evt_200_bin_wrnf_10m
['s2aspen_prob_10m_binOpt_wrnf', 'lc16_evt_200_bin_wrnf_10m', 'usfs_treemap16_balive_int_bin_wrnf_10m', 'usfs_itsp_aspen_ba_gt10_wrnf_10m']
Starting: usfs_treemap16_balive_int_bin_wrnf_10m
['s2aspen_prob_10m_binOpt_wrnf', 'lc16_evt_200_bin_wrnf_10m', 'usfs_treemap16_balive_int_bin_wrnf_10m', 'usfs_itsp_aspen_ba_gt10_wrnf_10m']
Starting: usfs_itsp_aspen_ba_gt10_wrnf_10m
NoData value: None
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Total elapsed time: 10.253333333333334 minutes.
